# Aula 1 - Parte Prática - Agentes & Ambientes

## Introdução

Nesse primeiro notebook iremos aprender sobre a API do [OpenAI Gym](http://gym.openai.com/) e começaremos a implementar alguns componentes de um agente de RL.
Ao final dessa parte prática teremos implementado o **ciclo de interação Agente-Ambiente** que nos permitirá futuramente treinar o agente e avaliar sua performance.

<img src="img/agent-env-loop.png" alt="Agent-Env Loop" style="width: 450px;"/>


### Objetivos:

- Relacionar os conceitos de Processos de Decisão Markovianos (MDPs) com os atributos e métodos de um ambiente definido com o OpenAI Gym;
- Familiarizar-se com os componentes básicos de um agente de RL;
- Implementar o ciclo de interação Agente-Ambiente; e
- Implementar um primeiro agente aleatório e avaliar sua performance

### Imports

> **Atenção:** não se esqueça de executar todos os `imports` necessários antes prosseguir com o tutorial. 

In [ ]:
import abc
import random

import gym

from utils.viz import plot_action_distribution, plot_episode_total_rewards

## 1. Ambientes no OpenAI Gym

Um ambiente no OpenAI Gym encapsula um simulador com o qual um agente pode interagir. Nesse simulador, a cada instante $t$ o agente deve escolhar uma ação $\mathbf{a}_t \in \mathcal{A}$ a ser executada. Ao receber essa ação, o ambiente tem seu estado $\mathbf{s}_t \in \mathcal{S}$ alterado para outro estado $\mathbf{s}_{t+1} \in \mathcal{S}$ e devolve para o agente uma observação (que pode ou não corresponder ao estado) e uma recompensa/punição $r_{t+1} \in \mathbb{R}$.

O pacote Gym conta com inúmeros [ambientes pré-programados](http://gym.openai.com/envs/) e prontos para serem usados para testar algoritmos de RL. Nessa parte prática, começaremos a explorar alguns ambientes mais simples a fim de nos familiarizarmos com os principais conceitos de modelagem da biblioteca Gym.

Para carregar um ambiente disponível no Gym basta chamar a função `gym.make` passando como argumento o identificador do ambiente:

In [ ]:
env = gym.make("MountainCar-v0")

> **Atenção**: antes de usar um ambiente do Gym com o qual você não está familiarizado pode ser útil ler a sua documentação online.
> Para o `MountainCar-v0` acesse o link [http://gym.openai.com/envs/MountainCar-v0/](http://gym.openai.com/envs/MountainCar-v0/).

Informações adicionais sobre o ambiente/simulador podem ser obtidas accessando o atributo `env.spec`. Embora não seja obrigatório, muitos ambientes do OpenAI Gym definem  metadados importantes relacionados à tarefa de RL. Em particular, `env.spec.max_episode_steps` define o número máximo de passos de decisão que um agente pode tomar em um episódio (i.e., o tamanho máximo de uma trajetória) e `env.spec.reward_threshold` define o valor mínimo de retorno (i.e., recompensa total) de um episódio para o qual a tarefa é considerada resolvida.

In [ ]:
print(f"MAX_EPISODE_STEPS = {env.spec.max_episode_steps}")
print(f"REWARD THRESHOLD = {env.spec.reward_threshold}")

### 1.1. Espaço de estados e ações

Todo agente de RL deve conhecer quais ações pode tomar no ambiente e também quais as características das variáveis de observações que aquele ambiente lhe disponibiliza. Para acessar o espaço de estados e ações, um ambiente do gym disponibiliza os atributos `env.observation_space` e `env.action_space`, respectivamente.

Note que essas informações serão importantes futuramente na definição das entradas e saídas das redes neurais artificiais que utilizaremos para representar a política $\pi_\theta$ do agente e também na criação de outros modelos.   

Todo `observation_space` tem associado seu tipo numérico (e.g., int, float,...) e as dimensões de uma observação. Além disso, é possível saber se o valor das variáveis observação são limitadas ou não, e se forem limitadas qual o valor mínimo e máximo.

In [ ]:
obs_space = env.observation_space

print(obs_space.dtype)
print(obs_space.shape)
print(obs_space.bounded_above, obs_space.low)
print(obs_space.bounded_below, obs_space.high)

Para o `MountainCar-v0` note que uma observação correponde a um vetor de números reais de tamanho 2 (i.e., um par de valores em ponto flutuante). Note que a primeira componente do vetor é limitada entre -1.2 e 0.6, e a segunda componente entre -0.07 e 0.07. 

Analogamente, todo `action_space` tem associado seu tipo numérico e as dimensões de uma ação:

In [ ]:
action_space = env.action_space

print(action_space.dtype)
print(action_space.shape)
print(action_space.n)

Note que para o `MountainCar-v0` o agente deverá escolher ações discretas (i.e., representadas por números inteiros). Observe também que o `shape==()` indica que uma ação é dada por um único escalar (e não um vetor como no caso do `observation_space`. Para acessar o número de possíveis valores da ação basta acessar o atributo `action_space.n`.

> **Atenção**: em algumas situações pode ser interessante ter acesso à amostras de observações e ações. Você pode usar os métodos `env.observation_space.sample()` e `env.action_space.sample()` para gerar aleatoriamente observações e ações direto do ambiente `env`. 

In [ ]:
obs_samples = [env.observation_space.sample() for _ in range(3)]
print(obs_samples)

In [ ]:
action_samples = [env.action_space.sample() for _ in range(5)]
print(action_samples)

### 1.2 Interface do Gym: métodos reset, step, render e close

Um objeto `env` do Gym fornece 4 métodos principais para interagir com o simulador: 
1. `reset` permite re-inicialiar o simulador para um de seus estados iniciais;
2. `step` se encarrega de executar uma ação no ambiente;
3. `render` visualiza graficamente o estado do agente; e
4. `close` libera recursos utilizados na simulação (por exemplo fecha a janela de visualização).

In [ ]:
for i in range(5):
    obs = env.reset()
    print(f"obs {i} = {obs}")

Note que a observação do estado inicial muda conforme o método `.reset()` é chamado. Isso se deve ao fato de que o estado inicial é definido como uma variável aleatória regida por uma distribuição inicial.

In [ ]:
obs = env.reset()

for i in range(5):
    action = env.action_space.sample()
    obs, reward, done, info = env.step(action)
    print(f"transition {i} = ({action}, {obs}, {reward})")

> **Atenção**: Você pode se familiarizar com os argumentos e retorno de ambos os métodos através de seus *docstrings* acessados via `?`.

In [ ]:
env.reset?

In [ ]:
env.step?

## 2. Agentes 

A fim de permitir a implementação do **ciclo de interação Agente-Ambiente**, um agente de RL deve ser capaz de escolher uma ação para cada observação recebida do ambiente e aprender (i.e., melhorar sua performance) a partir de suas experiências.

Nesse contexto, na classe abstrata `Agent` definimos a interface geral de um agente de RL. 

> **Atenção**: Familiarize-se com essa classe; todos os agentes definidos nessa aula e nas próximas deverão especializar (i.e., derivar ou sub-classear) essa interface geral.

In [ ]:
class RLAgent:
    """
    Classe abstrata que define a interface básica de um agente RL.

    Args:
        obs_space:     especificação do espaço de observações do ambiente.
        action_space:  especificação do espaço de ações do ambiente.
        config (dict): (opcional) configurações de hiper-parâmetros.
    """
    
    __metaclass__ = abc.ABCMeta

    def __init__(self, obs_space, action_space, config=None):
        self.obs_space = obs_space
        self.action_space = action_space
        self.config = config

    @abc.abstractmethod
    def act(self, obs):
        """
        Escolhe uma ação para ser tomada dada uma observação do ambiente.
        
        Args: 
            obs: observação do ambiente.
        
        Return:
            action: ação válida dentro do espaço de ações.
        """
        raise NotImplementedError

    @abc.abstractmethod
    def observe(self, obs, action, reward, next_obs, done):
        """
        Registra na memória do agente uma transição do ambiente.

        Args:
            obs:            observação do ambiente antes da execução da ação.
            action:         ação escolhida pelo agente.
            reward (float): escalar indicando a recompensa obtida após a execução da ação.
            next_obs:       nova observação recebida do ambiente após a execução da ação.
            done (bool):    True se a nova observação corresponde a um estado terminal, False caso contrário.

        Return:
            None
        """
        raise NotImplementedError

    @abc.abstractmethod
    def learn(self):
        """
        Método de treinamento do agente. A partir das experiências de sua memória,
        o agente aprende um novo comportamento.

        Args: 
            None

        Return:
            None
        """     
        raise NotImplementedError


### 2.1 Definindo um agente aleatório

Antes de finalmente definir o ciclo de interação agente-ambiente, vamos implementar um agente que escolhe ações aleatórias.

O agente `RandomPolicy` tem seu comportamento definido por uma política estocástica dada por uma distribuição uniforme sobre as ações válidas:

$$
\mathbf{a}_t \sim \pi(\cdot|\mathbf{s}) = \mathcal{Uniform}(\{ \mathbf{a} : \mathbf{a} \in \mathcal{A} \})~.
$$

Note que nesse ponto do curso, a implementação do agente aleatório é basicamente ilustrativa. No entanto, como veremos nas aulas seguintes, um agente que implementa uma política aleatória tem duas importantes funções:
1. servir de referência de performance final; e
2. guiar a inicialização de agentes de RL.

Em outras palavras, se um agente de RL após o treinamento não conseguir uma performance significativamente melhor do que aquela do agente aleatório, então muito provavelemente algo não está funcionando como deveria. Além disso, ao garantir que a inicialização de uma política induza um comportamente similar ao de um agente aleatório, não estaremos enviesando a exploração inicial do agente; o que poderia levar muito rapidamente para uma performance sub-ótima.

In [ ]:
class RandomPolicy(RLAgent):
    """
    Agente aleatório. Escolhe aleatoriamente uma ação independentemente
    da observação recebida do ambiente.
    
    Args:
        action_space:  especificação do espaço de ações do ambiente.
    """
    
    def __init__(self, observation_space, action_space, config=None):
        super(RandomPolicy, self).__init__(observation_space, action_space, config)

    def act(self, obs):
        """Retorna uma ação aleatória."""
        return self.action_space.sample()

    def observe(self, obs, action, reward, next_obs, done):
        """Ignora transições; i.e., um agente aletório não armazena experiências na memória."""
        pass

    def learn(self):
        """Um agente aletório não aprende; i.e., não melhora seu comportamento."""
        pass


Note que para uma mesma observação o agente `RandomPolicy` retorna diferentes ações a cada chamada:

In [ ]:
agent = RandomPolicy(env.observation_space, env.action_space)

obs = env.reset()
print([agent.act(obs) for _ in range(20)])

In [ ]:
plot_action_distribution(agent)

## 3. Ciclo de Interação Agente-Ambiente

Após entender a API do OpenAI Gym e se familiarizar com a interface geral de um agente de RL, estamos pronto para programar o ciclo de interação Agente-Ambiente.

Note que tanto o *treinamento* como a *avaliação de performance* de agentes de RL (baseados em simuladores) dependem da coleta de experiências a fim de estimar uma diferentes grandezas que são necessárias nos algoritmos de RL (e.g., retorno de episódios, gradientes de políticas, ...).

Esse é o objetivo principal do ciclo de interação com o ambiente: permitir ao agente explorar o ambiente e coletar dados para seu aprendizado.

> **Atenção**: Praticamente todos os pacotes de RL disponíveis implementam uma versão desse loop de interação. Dessa forma, independentemente se seu objetivo é desenvolver sua própria biblioteca de RL ou apenas re-utilizar código pré-existente, é importante entender os principais conceitos envolvidos!

Nessa seção testaremos a política aleatória em uma outra versão do ambiente MountainCar:

In [ ]:
env = gym.make("MountainCarContinuous-v0")
agent = RandomPolicy(env.observation_space, env.action_space)

---

**<font color="red">EXERCÍCIO-PROGRAMA 1:</font>**

Nesse exercício você deverá utilizar os métodos da API do Gym a fim de permitir que um agente de RL simule um episódio. Complete a função `sample_episode` com seu código e preste atenção para retornar as variáveis definidas na documentação. Caso necessário revise a <a href="/lab#1.2-Interface-do-Gym:-métodos-reset(),-step()-e-render()" target="_self">Seção 1.2</a>.

>**Atenção**: Para visualizar o episódio não se esqueça de chamar `env.render()` durante a simulação da trajetória. Use a flag `render` em um *if-statement* do Python para dinamicamente habilitar a visualização. Uma vez que a parte gráfica consome muito tempo, é comum desabilitar a renderização do ambiente durante o treinamento e avaliação de um agente de RL. Além disso, chame `env.close()` ao final do ciclo para fechar a janela de simulação.

In [ ]:
def sample_episode(agent, env, render=False):
    """
    Simula um episódio completo de interação do agente com o ambiente.
    
    Args:
        agent (RLAgent):       agente responsável por retornar ações.
        env (gym.Environment): simulador de ambiente do OpenAI Gym.
        render (bool):         (opcional) flag para habilitar a renderização do ambiente.
        
    Return:
        (total_reward, episode_length): retorno obtido pelo agente no episódio e
        número de passos de decisão realizados no episódio.
    """
    total_reward = 0.0
    episode_length = 0

    # SEU CÓDIGO AQUI ===================================
    obs = env.reset()
    
    done = False
    while not done:
        action = agent.act(obs)
        next_obs, reward, done, _ = env.step(action)
        
        total_reward += reward
        episode_length += 1
        
        if render:
            env.render()
            
        obs = next_obs
        
    env.close()
    
    # ===================================================

    return total_reward, episode_length


Execute o código abaixo para testar a sua implementação.

In [ ]:
total_reward, episode_length = sample_episode(agent, env, render=True)
print(f"return = {total_reward:.4f}, passos de decisão = {episode_length}\r", end="")

A menos que você tenha tido sorte na simulação, você deve ter observado que o carro nunca chega próximo à linha de chegada. :(

Não se preocupe vamos resolver isso nas próximas aulas!

---

Uma vez implementado a função `sample_episode` você pode amostrar diferentes trajetórias com a função `run` definida abaixo:

In [ ]:
def run(agent, env, num_episodes):
    episode_returns, episode_lengths = [], []

    for episode in range(num_episodes):
        total_reward, episode_length = sample_episode(agent, env)
    
        episode_returns.append(total_reward)
        episode_lengths.append(episode_length)

        if episode % 10 == 0:
            print(f"episode = {episode}, return = {total_reward:.4f}, length = {episode_length}\r", end="")

    return episode_returns, episode_lengths
    

Execute o código abaixo para simular `NUM_EPISODES` trajetórias:

In [ ]:
NUM_EPISODES = 200

episode_returns, episode_lengths = run(agent, env, NUM_EPISODES)

plot_episode_total_rewards(episode_returns)

> **Atenção**: se você obteve um pico de retorno com um valor positivo (i.e., *outlier*), execute novamente a simulação. Caso contrário, você deve ter obtido um retorno médio (vide linha vermelha) entre -34 e -33 para `NUM_EPISODES==200`.

---

**<font color="red">QUESTÕES:</font>**

1. Qual a diferença entre o ambiente `MountainCar-v0` utilizado como exemplo na <a href="/lab#1.-Ambientes-no-OpenAI-Gym" target="_self">Seção 1</a> e o ambiente `MountainCarContinuous-v0` que você acabou de simular?
2. Como você interpreta os gráficos acima `Episode Return` e `Episode Return (Histogram)` ? Como você explicaria essas variações ruidosas da recompensa total?
3. Se você executar a simulação várias vezes, obterá resultados ligeiramente diferentes? Ao que se deve essa incerteza nos resultados?
4. Você diria que o agente aleatório obteve uma boa performance? *Dica*: relacione os resultados obtidos com a especificação do ambiente `MountainCarContinuous-v0` (e.g., `env.spec`).
5. Note que durante a simulação o `Episode Length` se manteve constante ao longo dos episódios. O que isso significa do ponto de vista da tarefa de RL?

---
**<font color="blue">RESPOSTAS:</font>**

**Questão 1:**

A primeira diferença entre as versões discreta e contínua do MountainCar é com relação ao espaço de ações. Embora uma observação seja um par de número reais (e.g., `Box(2,)`) para ambos os ambientes, a representação de uma ação do agente é diferente dependendo da versão do ambiente. 

Para o `MountainCar-v0` as ações correspondeam aos números inteiros 0, 1 e 2 (pela documentação do problema disponível em http://gym.openai.com/envs/MountainCar-v0/ é possível inferir que esses 3 valores estão associadas à movimentação do carro para esquerda e direita, e um ação do tipo "ponto-morto").

Já para o `MountainCarContinuous-v0` a ação é um único escalar limitado entre -1.0 e 1.0. Novamente, através da simulação ambiente e também da documentação é possível inferir que essa ação está relacionada com a movimentação do carro (valores negativos para esquerda e valores positivos para a direita).

Você pode executar o código abaixo para relembrar os principais métodos da API do Gym para inspecionar as especificações dos espaços de observação e ação para cada ambiente:

In [ ]:
def print_env_info(env):

    def get_space_info(space):
        if isinstance(space, gym.spaces.Discrete):
            return f"{space}, n = {space.n}"
        elif isinstance(space, gym.spaces.Box):
            return f"{space}, low = {space.low}, high = {space.high}"
        else:
            raise ValueError(f"Space {space} is not a Box or a Dicrete space")
    
    print(f">> observation_space = {get_space_info(env.observation_space)}")
    print(f">> action_space = {get_space_info(env.action_space)}")

env_discrete = gym.make("MountainCar-v0")
env_continuous = gym.make("MountainCarContinuous-v0")

print("=== MountainCar-v0 ===")
print_env_info(env_discrete)
print()

print("=== MountainCarContinuous-v0 ===")
print_env_info(env_continuous)
print()

Além disso, na parte 1 do notebook observamos que a recompensa em cada passo de interação para a versão discreta do MountainCar é -1 e o `reward_threshold==-110.0`. Isso sugere que a tarefa de controlar o carro até subir a montanha está especificado como um problema de caminho mais curto, isto é, para um agente maximizar seu retorno médio ele deve levar menos tempo até chegar ao estado terminal do ambiente (e.g., a posição da bandeira no alto da montanha).

Por outro lado, na documentação do ambiente na página do OpenAI Gym para a versão contínua do MountainCar (http://gym.openai.com/envs/MountainCarContinuous-v0/) é mencionado que:
> "Here, the reward is greater if you spend less energy to reach the goal."

Isso sugere que além do incentivo para se alcançar o alto da montanha, o agente deve ser cauteloso ao "gastar" ações, isto é, deve tentar sempre que possível escolher ações de baixo valor absoluto (lembre-se que uma ação no MountainCarContinuous é um escalar entre -1.0 e 1.0).

Por fim, note que a documentação desses problemas do Gym não especifica em maiores detalhes a função de transição (i.e., como a ação de fato influencia no próximo estado) ou a função de recompensa (i.e, como o agente é penalizado ou recompensado por tomar uma ação). Isso é bastante comum em simuladores utilizados em aprendizado por reforço. Afinal, conforme discutido brevemente em aula, os algoritmos de aprendizado por reforço (model-free) não necessitam de acesso direto aos componentes do MDP, mas são capazes de aprender boas políticas apenas processando amostradas de observações e recompensas!

**Questão 2:**

Observamos no gráfico de `Episode Return` que não há nenhum padrão claro de melhoria ou degradação do desempenho do agente. Em outras palavras, o retorno de um episódio parece ser descrito por um ruído gaussiano em torno de um valor médio entre -33 e -34. Relembrando que esse resultado corresponde ao desempenho de um agente aleatório, é possível associar esse ruído simétrico à própria estocasticidade da política uniforme do agente.

**Questão 3:**

A incerteza (ou nesse contexto, a variabilidade dos resultados) se deve ao fato que a trajetória (i.e., uma sequência de pares (estado, ação)) induzida pela política do agente é de fato uma variável aleatória, e consequentemente, sempre que amostrada pode gerar resultados diferentes. Lembre-se também que a política que representa a tomada de decisão do agente é por si só estocástica. Nesse sentido, o agente mesmo com uma política pré-definida pode tomar ações diferentes em um mesmo estado revisitado em diferentes momentos.

**Questão 4:**

In [ ]:
print("=== MountainCar-v0 ===")
print(f">> reward_threshold = {env_discrete.spec.reward_threshold}, max_episode_steps = {env_discrete.spec.max_episode_steps}")
print()

print("=== MountainCarContinuous-v0 ===")
print(f">> reward_threshold = {env_continuous.spec.reward_threshold}, max_episode_steps = {env_continuous.spec.max_episode_steps}")
print()

Comparando os resultados obtidos com a especificação de retorno mínimo (i.e., `env.spec.reward_threshold`) fica claro que o agente aleatório não teve um bom desempenho. Isso era de certa forma uma conclusão esperado, visto o agente aleatório não aprende com seus erros e acertos (i.e., note que o método `learn` da classe `RandomAgent` se quer foi implementado!)

**Questão 5:**

O fato do `episode_length` não se alterar ao longo dos episódios é somente mais um indício de que o agente não está melhorando sua política. Essa é uma métrica importante para se monitorar ao longo do treinamento de um agente de RL, e é frequentemente usado para *debuggar* erros nos algoritmos.